## HPCCG
clang++-5.0.1 -g -O3 -funroll-loops -march=native    
  
(Haswell) Intel(R) Xeon(R) CPU E5-2699 v3 @ 2.30GHz  
`mulsd`: 5 cycles  
`movsd`: 3 cycles  
L1 Cache: 32 kB, 8 way, 64 sets, 64 B line size, **latency 4**, per core.    
L2 Cache: 256 kB, 8 way, 512 sets, 64 B line size, **latency 12**, per core.

|   | L1 Cache | L2 Cache | L3 Cache | DRAM |
|:---|:------:|:--------:|:----------:|:-----|
|Cache Lines / Cycle | 3.8886 | 1.2201 | 0.8750 | 0.4266 |
---
### Serial Run 100 100 100

|Scope|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | 
|:---|:---:|:---:|:---:|
|`HPC_sparsemv( )` | 80.6% CPU |    1.04 Ins / Cycle    | 7.5% L1 Data Cache Miss |
```c
|66|int HPC_sparsemv( HPC_Sparse_Matrix *A,                  
|67|		 const double * const x, double * const y)
|68|{
|69|
|70|  const int nrow = (const int) A->local_nrow;
|71|
|72|#ifdef USING_OMP
|73|#pragma omp parallel for
|74|#endif
|75|  for (int i=0; i< nrow; i++)
|76|    {
|77|      double sum = 0.0;
|78|      const double * const cur_vals = 
|79|     (const double * const) A->ptr_to_vals_in_row[i];
|80|
|81|      const int    * const cur_inds = 
|82|     (const int    * const) A->ptr_to_inds_in_row[i];
|83|
|84|      const int cur_nnz = (const int) A->nnz_in_row[i];
|85|
|86|      for (int j=0; j< cur_nnz; j++)                     //  9.9% CPUTIME(E)
```
|Scope|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | 
|:---|:---:|:---:|:---:|
|`sum += cur_vals[j]*x[cur_inds[j]]; ` | **58.8% CPU** |    1.09 Ins / Cycle    | 7.6% L1 Data Cache Miss |
```c
|87|        sum += cur_vals[j]*x[cur_inds[j]];             
|88|      y[i] = sum;
|89|    }
|90|  return(0);
|91|}
```




### 72 Thread Run 256 256 256

|Scope|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate |  L2 Data Cache Miss Rate |
|:---|:---:|:---:|:---:|:---:|
|`HPC_sparsemv( )` | 44.9% CPU |    0.13 Ins / Cycle    | 8.7% L1 Data Cache Miss |65.0% L2 Data Cache Miss |
```c
|66|int HPC_sparsemv( HPC_Sparse_Matrix *A,                  
|67|		 const double * const x, double * const y)
|68|{
|69|
|70|  const int nrow = (const int) A->local_nrow;
|71|
|72|#ifdef USING_OMP
|73|#pragma omp parallel for
|74|#endif
|75|  for (int i=0; i< nrow; i++)
|76|    {
|77|      double sum = 0.0;
|78|      const double * const cur_vals = 
|79|     (const double * const) A->ptr_to_vals_in_row[i];
|80|
|81|      const int    * const cur_inds = 
|82|     (const int    * const) A->ptr_to_inds_in_row[i];
|83|
|84|      const int cur_nnz = (const int) A->nnz_in_row[i];
|85|
|86|      for (int j=0; j< cur_nnz; j++)                    
```
|Scope|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | L2 Data Cache Miss Rate |
|:---|:---:|:---:|:---:|:---:|
|`sum += cur_vals[j]*x[cur_inds[j]]; ` | **38.4% CPU** |    0.12 Ins / Cycle    | 8.7% L1 Data Cache Miss | 64.8% L2 Data Cache Miss |
```c
|87|        sum += cur_vals[j]*x[cur_inds[j]];             
|88|      y[i] = sum;
|89|    }
|90|  return(0);
|91|}
```

When run one 1 thread HPCCG is bound by **memory latency** along with the depending **execution instructions**.  When run on Max threads HPCCG becomes increasingly bound by **memory**.

In [6]:
((15.7 *1e+09) / (2.3 *1e+09)) / 16

0.4266304347826087